In [ ]:
import sys
import os
sys.path.append("../Calculator/")
from PDF import PDF
from SpectraPlot import SpectraPlot
from multiprocessing import Pool

In [ ]:
#columns
wavelength=1
spectralRadiance=2

In [ ]:
#obligatory
title=""
name=""
fileList=[["file1"],["file2"]]
labelList=["label1","label2"]
desiredPlots=[
    {
        "xCol":wavelength,
        "yCol":spectralRadiance
    }
    ]
scaled=False

In [ ]:
#optional
customLims=False # better pick False running first Time
inputParameters={
                    "customFontsize":[12,12,8,10,10],
                    "labels":labelList,
                    "title":title
                }
inputParametersForScaled=inputParameters.copy()
inputParametersForScaled.update({
                    "scaleX":0.47,
                    "customFontsize":[10,10,4,6,6],
                    "ax2Labels":False,
                    "titleBool":False
                    })

In [ ]:
def initPlot(xCol=1, showColTup=(2,0), customInputParameters=None):
    if customInputParameters is not None:
        inputParameters.update(customInputParameters)
        inputParametersForScaled.update(customInputParameters)
    scPlot=None
    if customLims:
        plot=SpectraPlot(name, 
                  fileList,     
                  xCol=xCol,     
                  showColTup=showColTup,          
                  showColAxLim=yAxisLims, 
                  **inputParameters)
        if scaled:
            scPlot=SpectraPlot(name, 
                        fileList,      
                        xCol=xCol,     
                        showColTup=showColTup, 
                        xLimOrig=voltageLims[showColTup[0]], 
                        showColAxLim=yAxisLims, 
                        **inputParametersForScaled)
    else:
        plot=SpectraPlot(name, 
                  fileList,            
                  xCol=xCol,        
                  showColTup=showColTup,           
                  **inputParameters)
        if scaled:
            scPlot=SpectraPlot(name, fileList,
                        xCol=xCol, 
                        showColTup=showColTup,
                        **inputParametersForScaled)
    if scPlot is None:
        return [plot]
    return [plot,scPlot]

In [ ]:
def buildPlotList(desiredPlot):
    try:
        yCol2=desiredPlot["yCol2"]
    except KeyError:
        yCol2=0
    try:
        cusPara=desiredPlot["custom"]
    except KeyError:
        cusPara=None
    if desiredPlot["yCol"]==0 or desiredPlot["xCol"]==0:
        raise
    return initPlot(xCol=desiredPlot["xCol"], showColTup=(desiredPlot["yCol"],yCol2), customInputParameters=cusPara)

In [ ]:
def processPlotPair(plotpair):
    return [plot.doPlot() for plot in plotpair]

In [ ]:
#multithreaded
pool = Pool(os.cpu_count())
plotList=pool.map(buildPlotList,desiredPlots)
multiOutput=pool.map(processPlotPair,plotList)
pool.close()

In [ ]:
plots=[[plotOutput[0] for plotOutput in plotPair] for plotPair in multiOutput]
files=[[plotOutput[1] for plotOutput in plotPair] for plotPair in multiOutput]

In [ ]:
PDF(files[0][0], size=(700,700*3/4)) 